In [1]:
import numpy as np
import pandas as pd
import wfdb
import os
import scipy.signal as sgn
from tqdm import tqdm
from keras import models, layers, optimizers, regularizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from scipy.signal import cheby1, filtfilt

In [2]:
# File paths
train_files = [f"a{str(i).zfill(2)}" for i in range(1, 21)] + [f"b{str(i).zfill(2)}" for i in range(1, 6)] + [f"c{str(i).zfill(2)}" for i in range(1, 11)]
test_files = [f"x{str(i).zfill(2)}" for i in range(1, 36)]
base_path = "apnea-ecg/1.0.0/"
train_paths = [os.path.join(base_path, file) for file in train_files]
test_paths = [os.path.join(base_path, file) for file in test_files]


In [3]:

# Chebyshev Filter
def apply_chebyshev_filter(signal, lowcut=0.5, highcut=40, fs=100, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = cheby1(order, 0.5, [low, high], btype='band')
    return filtfilt(b, a, signal)

# Function to load and preprocess the ECG signal
def load_ecg_and_segment(file, segment_duration=10, fs=100):
    if not os.path.exists(f"{file}.hea") or not os.path.exists(f"{file}.dat"):
        print(f"File not found: {file}.hea or {file}.dat")
        return [], []
    
    record = wfdb.rdrecord(file)
    annotation = wfdb.rdann(file, 'apn')

    signal = record.p_signal[:, 0]
    filtered_signal = apply_chebyshev_filter(signal)

    segments = []
    labels = []
    samples_per_segment = segment_duration * fs

    for i, samp in enumerate(annotation.sample):
        start = max(samp - samples_per_segment // 2, 0)
        end = start + samples_per_segment
        if end > len(filtered_signal):
            break
        segment = filtered_signal[start:end]
        segments.append(segment)
        labels.append(1 if annotation.symbol[i] == 'A' else 0)
        
    return np.array(segments), np.array(labels)

# Load and prepare the full dataset
def prepare_data(paths):
    data, labels = [], []
    for path in tqdm(paths):
        segments, seg_labels = load_ecg_and_segment(path)
        if segments.size > 0:  
            data.extend(segments)
            labels.extend(seg_labels)
    return np.array(data), np.array(labels)


In [4]:
from sklearn.utils import shuffle

# Load training and validation data, and shuffle
X_train, y_train = prepare_data(train_paths)
X_test, y_test = prepare_data(test_paths)

# Shuffle the data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Print the number of segments loaded for verification
print(f"Training segments: {len(X_train)}, Test segments: {len(X_test)}")

# Reshape data for model input format
X_train = X_train[..., np.newaxis]  # Adding channel dimension
X_test = X_test[..., np.newaxis]

print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Test data shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:04<00:00,  8.27it/s]

Training segments: 17045, Test segments: 17268
Training data shape: (17045, 1000, 1)
Training labels shape: (17045,)
Test data shape: (17268, 1000, 1)
Test labels shape: (17268,)


In [5]:
from sklearn.model_selection import StratifiedKFold

# Splitting the training data into train and validation sets with StratifiedKFold
strat_kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
train_indices, val_indices = next(strat_kfold.split(X_train, y_train))

# Creating training and validation sets
X_ctrain, X_val = X_train[train_indices], X_train[val_indices]
y_ctrain, y_val = y_train[train_indices], y_train[val_indices]

# Print shapes of the resulting sets for verification
print("Train data shape:", X_ctrain.shape)
print("Validation data shape:", X_val.shape)
print("Train labels shape:", y_ctrain.shape)
print("Validation labels shape:", y_val.shape)

Train data shape: (14610, 1000, 1)
Validation data shape: (2435, 1000, 1)
Train labels shape: (14610,)
Validation labels shape: (2435,)


In [6]:
from tensorflow.keras import models, layers, regularizers, optimizers

def build_cnn_lstm_model(input_shape):
    # Input layer
    input_layer = layers.Input(shape=input_shape)
    x_bn = layers.BatchNormalization()(input_layer)
    
    # First branch: Conv1D with kernel size 125 and MaxPooling with L2 regularization
    branch1 = layers.Conv1D(24, kernel_size=125, strides=1, activation='relu', 
                            padding='same', kernel_regularizer=regularizers.l2(0.001))(x_bn)
    branch1 = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(branch1)
    
    # Second branch: Conv1D with kernel size 15 and MaxPooling with L2 regularization
    branch2 = layers.Conv1D(24, kernel_size=15, strides=1, activation='relu', 
                            padding='same', kernel_regularizer=regularizers.l2(0.001))(x_bn)
    branch2 = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(branch2)
    
    # Third branch: Conv1D with kernel size 5 and MaxPooling with L2 regularization
    branch3 = layers.Conv1D(24, kernel_size=5, strides=1, activation='relu', 
                            padding='same', kernel_regularizer=regularizers.l2(0.001))(x_bn)
    branch3 = layers.MaxPooling1D(pool_size=2, strides=1, padding='same')(branch3)
    
    # Concatenate the outputs of the three branches
    concatenated = layers.Concatenate(axis=-1)([branch1, branch2, branch3])
    
    # Additional MaxPooling after concatenation
    x = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')(concatenated)
    
    # Adjust the Conv1D layer to match dimensions for addition
    conv_adjusted = layers.Conv1D(72, kernel_size=3, strides=1, activation='relu', 
                                  padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Add()([x, conv_adjusted])
    
    # Fully Connected and Dropout layers with L2 regularization
    x = layers.Dense(48, activation='leaky_relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)  # Dropout for regularization
    
    # Global Pooling layer
    x = layers.GlobalAveragePooling1D()(x)
    
    # LSTM layer
    # x = layers.Reshape((1, -1))(x)  # Reshape for LSTM input
    # x = layers.LSTM(64, kernel_regularizer=regularizers.l2(0.01))(x)
    
    # Output layer
    output_layer = layers.Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(0.001))(x)
    
    # Define the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    return model


In [8]:

# Compile the model with specified learning rate
input_shape = (1000, 1)  # Adjust as per 10-second segments at 100 Hz sampling rate
model = build_cnn_lstm_model(input_shape)
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Model summary for verification
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1000, 1)]            0         []                            
                                                                                                  
 batch_normalization_1 (Bat  (None, 1000, 1)              4         ['input_2[0][0]']             
 chNormalization)                                                                                 
                                                                                                  
 conv1d_4 (Conv1D)           (None, 1000, 24)             3024      ['batch_normalization_1[0][0]'
                                                                    ]                             
                                                                                            

In [9]:
# Callbacks
# Define the model checkpoint
checkpoint_path = "best_model.h5"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

# Train the model
history = model.fit(X_ctrain, y_ctrain, 
                    validation_data=(X_val, y_val), 
                    epochs=30, 
                    batch_size=16,
                    class_weight=class_weights_dict,
                    callbacks=[checkpoint, reduce_lr, early_stopping]
                   )

Epoch 1/30


914/914 [==============================] - ETA: 0s - loss: 0.6799 - accuracy: 0.6761
Epoch 1: val_loss improved from inf to 0.68605, saving model to C:/Users/abbas/BAU/11Fall 2024/FYP2/apnea-ecg/1.0.0/Model_CheckPoint_draft_3\best_model.h5
914/914 [==============================] - 44s 47ms/step - loss: 0.6799 - accuracy: 0.6761 - val_loss: 0.6861 - val_accuracy: 0.6559 - lr: 0.0010
Epoch 2/30
  1/914 [..............................] - ETA: 41s - loss: 0.7517 - accuracy: 0.7500

C:\Users\abbas\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


913/914 [============================>.] - ETA: 0s - loss: 0.5874 - accuracy: 0.7362
Epoch 2: val_loss improved from 0.68605 to 0.68332, saving model to C:/Users/abbas/BAU/11Fall 2024/FYP2/apnea-ecg/1.0.0/Model_CheckPoint_draft_3\best_model.h5
914/914 [==============================] - 42s 46ms/step - loss: 0.5875 - accuracy: 0.7362 - val_loss: 0.6833 - val_accuracy: 0.7819 - lr: 0.0010
Epoch 3/30
913/914 [============================>.] - ETA: 0s - loss: 0.5448 - accuracy: 0.7499
Epoch 3: val_loss did not improve from 0.68332
914/914 [==============================] - 42s 46ms/step - loss: 0.5448 - accuracy: 0.7499 - val_loss: 0.7490 - val_accuracy: 0.3873 - lr: 0.0010
Epoch 4/30
914/914 [==============================] - ETA: 0s - loss: 0.5300 - accuracy: 0.7598
Epoch 4: val_loss improved from 0.68332 to 0.63249, saving model to C:/Users/abbas/BAU/11Fall 2024/FYP2/apnea-ecg/1.0.0/Model_CheckPoint_draft_3\best_model.h5
914/914 [==============================] - 42s 46ms/step - loss: 0

In [10]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict on the test set
y_test_pred = np.argmax(model.predict(X_test), axis=1)

# Compute confusion matrix and evaluation metrics
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
TAC = (tp + tn) / (tp + tn + fp + fn) * 100
sensitivity = tp / (tp + fn) * 100
specificity = tn / (tn + fp) * 100
PPV = tp / (tp + fp) * 100
NPV = tn / (tn + fn) * 100
kappa = cohen_kappa_score(y_test, y_test_pred)

# Display metrics
print(f"Total Accuracy (TAC): {TAC:.2f}%")
print(f"Sensitivity (SE): {sensitivity:.2f}%")
print(f"Specificity (SP): {specificity:.2f}%")
print(f"Positive Predictive Value (PPV): {PPV:.2f}%")
print(f"Negative Predictive Value (NPV): {NPV:.2f}%")


Test Accuracy: 65.00%
540/540 [==============================] - 9s 17ms/step
Total Accuracy (TAC): 65.00%
Sensitivity (SE): 75.60%
Specificity (SP): 58.52%
Positive Predictive Value (PPV): 52.69%
Negative Predictive Value (NPV): 79.70%
